In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import keras
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import gensim
from gensim import corpora,models,utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

Using TensorFlow backend.
C:\Users\Scott Sun\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# xml parsing
import xml.etree.ElementTree as ET

tree = ET.parse("./COLIEE2018_CaseLaw_Training_Data/IR/IR.xml")
root = tree.getroot()
print(root.tag)

cases_noticed_list = []
for node in root.iter('cases_noticed'):
    if node.text:
        cases_noticed_list.append(list(node.text.split(',')))
    else:
        cases_noticed_list.append([0])
# cases_noticed_list

COLIEE_2018


word-vec

In [3]:
target_article_list=[]
candidate_article_list_l=[]
for no in range(1,286):
        
        if no<10:
            nos = '00'+str(no)
        elif no<100:
            nos = '0'+str(no)
        else:
            nos = str(no)
        target_articlef = open('./COLIEE2018_CaseLaw_Training_Data/IR/data/'+nos+'/fact.txt',encoding='utf-8')
        target_article = target_articlef.read().replace('\n','').replace('[','').replace(']','')
        target_article_list.append(target_article)
        
        candidate_article_list = []
        for i in range(200):
            if i<10:
                index = '00'+str(i)
            elif i<100:
                index = '0'+str(i)
            else:
                index = str(i)
            candidate_articlef = open('./COLIEE2018_CaseLaw_Training_Data/IR/data/'+nos+'/candidates/'+index+'.txt',encoding='utf-8')
            candidate_article = candidate_articlef.read().replace('\n','').replace('[','').replace(']','')
            candidate_article_list.append(candidate_article)
        candidate_article_list_l.append(candidate_article_list)

In [4]:
model_doc = Doc2Vec(vector_size=50, min_count=5, epochs=55)

In [11]:
target_train_list=[]
tn = 1
for ta in target_article_list:
    target_train_list.append(models.doc2vec.TaggedDocument(utils.simple_preprocess(ta), 'target'+str(tn)))
    tn+=1
#     print(tn)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [6]:
can_train_list=[]
casen = 1
for candidate_article_list in candidate_article_list_l:
    cn = 0
    for candidate_article in candidate_article_list:
        can_train_list.extend(models.doc2vec.TaggedDocument(utils.simple_preprocess(candidate_article), 'case'+str(casen)+'candidate'+str(cn)))
        cn+=1
    casen+=1
    print(casen)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [6]:
def read_corpus_target(tokens_only=False):
    for index, content in zip(range(1,286),target_article_list):
            if tokens_only:
                yield utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield models.doc2vec.TaggedDocument(utils.simple_preprocess(content), 'target'+str(index))

In [7]:
target_train_list2 = list(read_corpus_target(tokens_only=False))

In [8]:
def read_corpus_candidate(tokens_only=False):
    for i in range(0,285):
        for index, content in zip(range(1,286),candidate_article_list_l[i]):
            if tokens_only:
                yield utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield models.doc2vec.TaggedDocument(utils.simple_preprocess(content), 'case'+str(i)+'candidate'+str(index))

In [10]:
can_train_list2 = list(read_corpus_candidate(tokens_only=False))

MemoryError: 

In [9]:
train_list = target_train_list2+can_train_list2

In [25]:
model_doc.build_vocab(train_list)

In [ ]:
%time model_doc.train(train_list, total_examples=model_doc.corpus_count, epochs=model_doc.epochs)

In [ ]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")
model_doc.save(fname)

after training

In [4]:
model_doc_train = Doc2Vec.load("my_doc2vec_model")

In [5]:
vectors1 = dict()
for i in train_list:
    name = i[1]
    vectors1[name]=model_doc_train.infer_vector(i[0])

NameError: name 'train_list' is not defined

In [12]:
data = pd.DataFrame.from_dict(vectors1).transpose()
data.shape

(57285, 50)

In [14]:
import random
#split
rem = random.sample(range(286),86)
# rem
data.to_csv('data_t1.csv')

In [74]:
# cases_noticed_list[0]
# can_train_list2[0]
# candidate_article_list_l[0][18]
# cases_noticed_list[53]
# len(score)
# len(dataset)
# len(pred)
# for rpa in cases_noticed_list[20]:
#     string_l = [target_article_list[20]+candidate_article_list_l[20][int(rpa)]]
#     count = CountVectorizer(min_df=0,ngram_range=(2,3))
#     count.fit(string_l)
#     count1 = count.transform([target_article_list[20]]).todense()
#     A = count.transform([candidate_article_list_l[20][int(rpa)]]).todense()
#     B = np.matmul(count1,A.T).T
#     C = np.sum(A) + np.sum(count1)-B
#     D = B / C
# cases_noticed_list[20]
# rem


3701

In [98]:
#training
import random
pred = []
dataset = dict()
score = []
for index in range(len(cases_noticed_list)):
    if index == 53:
        continue
    if index in rem:
#         print(index)
        continue
    n=1
    case = 'target'+str(index+1)
    seq = []
    for i in range(200):
        seq.append(str(i))
    for rpa in cases_noticed_list[index]:
#         rpa = str(rpa).replace(' ','')
#         print(rpa)
        seq.remove(rpa)
        para = 'case'+str(index) + 'candidate'+str(int(rpa)+1)
        temp = data.loc[case,:].tolist()
        for i in data.loc[para,:].tolist():
            temp.append(i)
        dataset[case+'-'+str(n)]=temp
        n+=1
        pred.append(1)
        
        string_l = [target_article_list[index]+candidate_article_list_l[index][int(rpa)]]
        
        count = CountVectorizer(min_df=0,ngram_range=(2,3),stop_words='english')
        count.fit(string_l)
        count1 = count.transform([target_article_list[index]]).todense()
        A = count.transform([candidate_article_list_l[index][int(rpa)]]).todense()
        B = np.matmul(count1,A.T).T
        C = np.sum(A, axis = 1) + np.sum(count1)-B
        D = B / C
        score.append(D[0,0])
#     print(index,':',len(dataset),',',len(score))    
    candi = random.sample(seq,len(cases_noticed_list[index]))
    for num in candi:
        para = 'case'+str(index) + 'candidate'+str(int(num)+1)
        temp = data.loc[case,:].tolist()
        for i in data.loc[para,:].tolist():
            temp.append(i)
        dataset[case+'-'+str(n)]=temp
        n+=1
        pred.append(0)
        
        string_l = [target_article_list[index]+candidate_article_list_l[index][int(num)]]
        count = CountVectorizer(min_df=0,ngram_range=(2,3),stop_words='english')
        count.fit(string_l)
        count1 = count.transform([target_article_list[index]]).todense()
        A = count.transform([candidate_article_list_l[index][int(num)]]).todense()
        B = np.matmul(count1,A.T).T
        C = np.sum(A, axis = 1) + np.sum(count1)-B
        D = B / C
        score.append(D[0,0])
    if len(dataset) != len(score):
        print(index,':',len(dataset),',',len(score))
#     print(index)

In [91]:
data2 = pd.DataFrame.from_dict(dataset).transpose()
data2['score'] = score
data2.to_csv('data2_t1.csv')
# data2

In [92]:
train_x = data2.values

In [94]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=50, random_state=0, n_estimators=100)
rf.fit(train_x,pred)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [95]:
result_rf = []
for index in range(len(cases_noticed_list)):
    result_t=[]
    if index not in rem:
#         print(index)
        continue
    case = 'target'+str(index+1)
    for rpa in range(len(candidate_article_list_l[index])):
        temp = data.loc[case,:].tolist()
        para = 'case'+str(index) + 'candidate'+str(rpa+1)
        for i in data.loc[para,:].tolist():
            temp.append(i)
#         print(rf.predict_proba([temp])[0][0])

        string_l = [target_article_list[index]+candidate_article_list_l[index][rpa]]
        count = CountVectorizer(min_df=0,ngram_range=(2,3))
        count.fit(string_l)
        count1 = count.transform([target_article_list[index]]).todense()
        A = count.transform([candidate_article_list_l[index][rpa]]).todense()
        B = np.matmul(count1,A.T).T
        C = np.sum(A, axis = 1) + np.sum(count1)-B
        D = B / C
        temp.append(D[0,0])
        result_t.append(rf.predict_proba([temp])[0][0])
    result_rf.append(np.argsort(result_t))


In [99]:
result_rf[1]

array([142, 191,  41,  32, 143, 190,  99, 194, 140, 150,  21, 175,  58,
         1, 122, 182, 131,  55, 129,  16, 146, 149, 162, 197, 172,  59,
        70,  76,  80,  25, 126, 137, 145, 195,  98, 117,  84, 152,  75,
       193, 111, 102, 199,  31,  12, 151,  29, 158,  92,  49, 183,  67,
        60,  18,  57,  72,  79,  63, 156, 104, 135, 100, 101, 105, 153,
        53,  86, 176,  34,  87, 164,  78,  11,  77,  95, 106, 170, 148,
        44, 181,  48, 180, 138, 110,  54, 127,   6, 192,  28,  64, 173,
       115,  69, 124, 155, 136, 198,  43, 133,  94, 179,  71,  73, 166,
       177,  81, 174,  85,  47,  46,  33,  52, 128, 125,  51, 123, 141,
        26,   5,  10, 171,  35, 107,  90,  61,  88, 157, 178,  19,  15,
       167, 134, 160, 159, 113, 184,  93,  40,  96,  56,  30,  42, 196,
        37,  66, 188,   4,  74, 112, 185,  17, 144,   7, 139,  45, 161,
        83,  38, 168,  39, 165,  50, 163,  91,  97, 154, 109,  13, 130,
        14, 120, 189,  23, 108, 187,   3, 147, 114,  20,   8,  8

In [101]:
for num in range(1,51):
    precision = []
    recall = []
    for j in range(len(result_rf)):
        r = result_rf[j][-num:]
        prtop = 0
        for re in r:
            if str(re+1) in cases_noticed_list[rem[j]]:
                prtop+=1
        precision.append(prtop)
        recall.append(len(cases_noticed_list[rem[j]]))
    
    pre = 0.0
    rec = 0.0
    for i in range(len(precision)):
        pre+= precision[i]/num
        rec+= precision[i]/recall[i]
        
    pre = pre/181
    rec = rec/181
    F2 = 5*pre*rec/(4*pre+rec)
    
    print('num of prediction: ',num)
    print('precision: ',pre)
    print('recall: ',rec)
    print('F2-measure: ',F2)

num of prediction:  1
precision:  0.022099447513812154
recall:  0.002212336466480113
F2-measure:  0.002697900085822389
num of prediction:  2
precision:  0.024861878453038673
recall:  0.00528006612624234
F2-measure:  0.0062673252992125365
num of prediction:  3
precision:  0.025782688766114177
recall:  0.008176433111007115
F2-measure:  0.009469758133936827
num of prediction:  4
precision:  0.024861878453038673
recall:  0.011076177868850156
F2-measure:  0.012457717603859081
num of prediction:  5
precision:  0.026519337016574593
recall:  0.01813214041021128
F2-measure:  0.01935650602333743
num of prediction:  6
precision:  0.027624309392265192
recall:  0.020253895047751238
F2-measure:  0.021395602929010778
num of prediction:  7
precision:  0.025256511444356748
recall:  0.021504258525506397
F2-measure:  0.02216278473483135
num of prediction:  8
precision:  0.024171270718232045
recall:  0.02219757452593972
F2-measure:  0.02256609983718827
num of prediction:  9
precision:  0.02209944751381216